# Pre-requisite (Optional):












To chat with your private google documents, perform the following steps to authorize access to the google document.

##Create a service account:
Follow the Steps at this link: https://support.google.com/a/answer/7378726?hl=en

*Note: Enable 'Google Docs API' if you want to read from google documents *

upload the json file that contains credentials to a secure google drive location

##Share the Google Doc with the service account:
Steps:

Go to your Google Doc (with the DOCUMENT_ID you're using).
Click the "Share" button in the top right corner.
In the "Share with people and groups" field, enter the email address of your service account (found in your credentials.json file).
Ensure the service account has "Viewer" or "Editor" permissions.
Click "Send".

# Setup

This is necessary for setting up the necessary python environment. Install required libraries to run code that depends on these libraries.

Sentence-transformers: This package is likely used for working with text data, specifically for generating sentence embeddings (numerical representations of sentences).

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install -q sentence-transformers
!pip install -q numpy
!pip install -q scipy
!pip install -q openai

# Step-1: Get the data

Use any one of the options based on your setup.



##Option-1: If you have already setup google service account, then you can interact with your google drive documents using the following code


This will prompt you to authroize Colab to access your Google Drive. Follow the instructions to complete the authroization

This code is designed to fetch and print the text content of a Google Doc. It iterates through the structure of the document's content, specifically looking for paragraphs and text runs within them. It appends the text from each text run to the doc variable, effectively building a string containing the entire text of the document.

In [ ]:
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.colab import drive
import os

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/documents.readonly']

# This code snippet mounts your Google Drive to the /content/drive directory in the Colab environment,
# allowing the script to access files within your Drive.
drive.mount('/content/drive')

# REPLACE THE FOLLOWING PATH WITH YOUR OWN CREDENTIALS FILE LOCATION
SERVICE_ACCOUNT_FILE = '/content/drive/My Drive/auth/credentials.json'

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)


service = build('docs', 'v1', credentials=credentials)

# Replace with the document ID of your Google Doc
# UPDATE WITH YOUR OWN document Id
DOCUMENT_ID = '1SLeIqkuF71IiNinw0pYXFvgIIgpW-DZzR7MWiUYdrhg'

# Get the document content
document = service.documents().get(documentId=DOCUMENT_ID).execute()

# Extract the text content
doc = ''
for element in document.get('body').get('content'):
    if 'paragraph' in element:
        for content in element.get('paragraph').get('elements'):
            if 'textRun' in content:
                doc += content.get('textRun').get('content')

print(doc)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
STORY


In the heart of the Whispering Woods, where ancient trees whispered secrets to the wind, lived a young elf named Elara. With her fiery red hair and emerald green eyes, she was as curious as a kitten, always yearning for adventure. One day, while exploring the woods, she stumbled upon a hidden cave, its entrance guarded by a wise old owl named Orion.


Orion, with his knowing gaze and feather-soft voice, revealed a secret map to Elara. It led to the Enchanted Forest, a place of wonder and magic, where a legendary crystal, the Heartstone, was said to grant eternal youth. Intrigued and excited, Elara decided to embark on this magical journey.


She invited her best friend, a mischievous sprite named Zephyr, to join her. Zephyr, with his playful spirit and quick wit, was the perfect companion for such an adventure. Together, they set off, following the ma

## Option-2: Alternatively, interact with the locally specified data source as below.

In [ ]:
doc = repr("""
STORY


In the heart of the Whispering Woods, where ancient trees whispered secrets to the wind, lived a young elf named Elara. With her fiery red hair and emerald green eyes, she was as curious as a kitten, always yearning for adventure. One day, while exploring the woods, she stumbled upon a hidden cave, its entrance guarded by a wise old owl named Orion.


Orion, with his knowing gaze and feather-soft voice, revealed a secret map to Elara. It led to the Enchanted Forest, a place of wonder and magic, where a legendary crystal, the Heartstone, was said to grant eternal youth. Intrigued and excited, Elara decided to embark on this magical journey.


She invited her best friend, a mischievous sprite named Zephyr, to join her. Zephyr, with his playful spirit and quick wit, was the perfect companion for such an adventure. Together, they set off, following the map's cryptic clues.


Their journey was filled with challenges and wonders. They crossed treacherous rivers, scaled towering mountains, and evaded the watchful eyes of forest creatures. Along the way, they met a grumpy troll named Grumpkin, who initially tried to hinder their progress. However, with Elara's kindness and Zephyr's humor, they managed to win over Grumpkin's heart, and he even became their guide.


Finally, they reached the Enchanted Forest, a breathtaking realm of vibrant colors and ethereal beauty. The Heartstone, a radiant crystal pulsing with life, was nestled deep within the forest's heart. As they approached the crystal, a powerful energy surged through them, and they felt a sense of peace and rejuvenation.


With the Heartstone's power, Elara and Zephyr returned to the Whispering Woods, sharing their newfound wisdom and energy with the forest's inhabitants. The once gloomy Grumpkin became a cheerful guardian of the woods, and the Whispering Woods thrived, its magic renewed. And so, the adventure of Elara, Zephyr, and Grumpkin ended with a happy ending, forever etched in the annals of forest lore."""
)

# Step-2: Chunk the data


This line of code takes the entire text content from the Google Doc (stored in doc) and breaks it up into separate paragraphs whenever it finds two newline characters in a row. The resulting paragraphs are then stored as a list in the paragraphs variable.

In [ ]:
paragraphs = doc.split('\n\n')
print(paragraphs)

['STORY', '\nIn the heart of the Whispering Woods, where ancient trees whispered secrets to the wind, lived a young elf named Elara. With her fiery red hair and emerald green eyes, she was as curious as a kitten, always yearning for adventure. One day, while exploring the woods, she stumbled upon a hidden cave, its entrance guarded by a wise old owl named Orion.', '\nOrion, with his knowing gaze and feather-soft voice, revealed a secret map to Elara. It led to the Enchanted Forest, a place of wonder and magic, where a legendary crystal, the Heartstone, was said to grant eternal youth. Intrigued and excited, Elara decided to embark on this magical journey.', "\nShe invited her best friend, a mischievous sprite named Zephyr, to join her. Zephyr, with his playful spirit and quick wit, was the perfect companion for such an adventure. Together, they set off, following the map's cryptic clues.", "\nTheir journey was filled with challenges and wonders. They crossed treacherous rivers, scaled 

# Step-3: Load a pre-trained embedding model

Responsible for importing the necessary tools and loading a pre-trained sentence embedding model called "gte-base-en-v1.5". This model will likely be used in subsequent code to generate numerical representations of sentences, which can be used for tasks like semantic similarity comparison or text classification.

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)
docs_embed = model.encode(paragraphs, normalize_embeddings=True)
docs_embed.shape # shape of each vector in the index

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

(7, 768)

In [ ]:
docs_embed[0]

array([ 3.89194675e-03, -1.02055445e-02,  5.75161939e-05,  6.02248423e-02,
        5.42831756e-02, -2.52313279e-02,  2.24598832e-02,  9.38612297e-02,
        1.63434260e-02,  3.14385779e-02,  1.17918840e-02, -3.74607295e-02,
        5.97808063e-02,  8.39209259e-02,  5.32138757e-02, -8.08254443e-03,
       -3.83747257e-02, -4.25602719e-02, -4.10649180e-02,  2.94339936e-02,
        6.80894917e-03, -4.58970033e-02, -5.12931794e-02, -7.14389887e-03,
       -1.18238563e-02, -3.55313458e-02,  8.01298860e-03,  2.87530329e-02,
       -1.63872726e-02, -3.10424119e-02, -2.09486000e-02, -2.87608686e-03,
        1.02233896e-02,  1.47648705e-02,  3.40597294e-02, -1.39749674e-02,
        4.55656787e-03,  2.01155003e-02,  5.56692947e-03, -3.44361141e-02,
       -8.56262725e-03,  3.93640548e-02,  1.52302979e-04, -2.18384378e-02,
        2.59840488e-02, -4.17519687e-03, -1.10182557e-02,  2.46889889e-02,
        5.42182997e-02, -3.71034071e-02, -6.28011078e-02,  3.80584449e-02,
        1.67084183e-03, -

# Step-4: User Query

In [ ]:
query = "who revealed secret map to Elara?"

# Step-5: Embed the user query

Imagine you want to compare the meaning of two sentences. This code takes the user's query (a sentence) and transforms it into a special numerical code (embedding) that captures its meaning. This code is then stored in query_embed. The normalize_embeddings=True part makes sure this numerical code is in a standard format, which helps in comparing it to other codes representing different sentences.

Overall, this line is crucial for converting the user's text query into a format that can be used for further processing and comparison with the document's content.

In [ ]:
query_embed = model.encode(query, normalize_embeddings=True) # Embed the query string to obtain a query vector
query_embed.shape

(768,)

# Step-6: Search Relevant Source Information

This calculates the similarity between the user's query and each chunk of the document by performing a dot product between their respective embeddings. The higher the dot product, the more similar the query and the document chunk are considered to be. The resulting similarities variable will hold an array of similarity scores, one for each chunk of the document.

Imagine you have a list of scores for different documents, and you want to find the top 3 documents with the highest scores. This line of code does exactly that. It sorts the scores, picks the top 3, and gives you the positions (indices) of those documents in the original list. The top_3_idx variable then holds these positions, allowing you to easily access the most relevant document chunks

In [ ]:
import numpy as np
similarities = np.dot(docs_embed, query_embed.T)
similarities.shape # use the query vector to find the most similar document to the query

(7,)

In [ ]:
similarities

array([0.35064158, 0.31032276, 0.523458  , 0.23463485, 0.34480548,
       0.2323069 , 0.4556533 ], dtype=float32)

In [ ]:
top_3_idx = np.argsort(similarities, axis=0)[-3:][::-1].tolist()
top_3_idx

[2, 6, 0]

# Step-7: Retrieve relevant chunks + user query

This code takes the most similar document chunks, formats them for readability, prints them to the output, and stores them in a single string variable called CONTEXT for potential later use.

In [ ]:
CONTEXT = [paragraphs[idx] for idx in top_3_idx] # return the top 3 most similar docs
print(CONTEXT)

['\nOrion, with his knowing gaze and feather-soft voice, revealed a secret map to Elara. It led to the Enchanted Forest, a place of wonder and magic, where a legendary crystal, the Heartstone, was said to grant eternal youth. Intrigued and excited, Elara decided to embark on this magical journey.', "\nWith the Heartstone's power, Elara and Zephyr returned to the Whispering Woods, sharing their newfound wisdom and energy with the forest's inhabitants. The once gloomy Grumpkin became a cheerful guardian of the woods, and the Whispering Woods thrived, its magic renewed. And so, the adventure of Elara, Zephyr, and Grumpkin ended with a happy ending, forever etched in the annals of forest lore.\n", 'STORY']


In [ ]:
query = "who revealed secret map to Elara?"

# Step-8: Augment input for the LLM

This code is constructing a structured input for an LLM, providing it with the context and question necessary to generate a relevant response. It's like giving a detailed briefing to someone before asking them a question, ensuring they have all the information they need.

In [ ]:
prompt = f"""
use the following CONTEXT to answer the QUESTION at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

CONTEXT: {CONTEXT}
QUESTION: {query}

"""

# Step-9: Query the LLM with the Augmented Input

This code snippet prepares the environment to make calls to the OpenAI API by importing the necessary library (openai) and attempting to retrieve the API key securely from the user's Colab data. This setup enables the user to interact with OpenAI's language models and leverage their capabilities in their code.

This also sends your carefully constructed prompt (containing the context and your question) to the "gpt-4o" model on OpenAI's servers. The model processes the information and generates a response. This response is then stored in the response variable for you to use in your program. It's like sending a text message to a very smart AI assistant and waiting for their reply.

In [ ]:
# prompt: write python code to make calls to openai api
from google.colab import userdata
userdata.get('openai')

from openai import OpenAI
client = OpenAI(api_key=userdata.get('openai'))

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": prompt},
  ]
)

# Step-10: Generate a RAG response

This line of code takes the response from the LLM, navigates through its structure to find the generated answer, and then prints it out for you to read.

In [ ]:
print(response.choices[0].message.content)

Orion revealed the secret map to Elara.
